# 신경망 모델 만들기
- 신경망은 데이터 연산을 수행하는 layers와 module로 구성된다.
- 신경망은 다른layer(모듈)로 구성된 하나의 모듈이다. 
- 이렇게 중첩된구조는 복잡한 아키텍처를쉽게 구축하고 관리하는데 좋다


In [11]:
# fashionMNIST 분류하는 신경망 만들어 보기

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = ('cuda' if torch.cuda.is_available()
          else 'cpu')

print(f'using {device}')

using cpu


In [12]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        # 인잇에서는 신경망의 레이어를 정의한다
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear (512, 10),
        )
    
    # 포워드에서는 입력받은 데이터들에 대한 연산을 수행한다
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    

In [13]:
# NeuralNetwork의 인스턴스를 생성해준다
model = NeuralNetwork().to(device)

# 모델 구조 확인
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [45]:
X = torch.rand(1, 28, 28, device = device) # 랜덤데이터 생성
logits = model(X)
print(logits) # 로짓데이터이다 이를 소프트맥스함수에 통과시켜 확률로 확인한다.

pred_probab = nn.Softmax(dim=1)(logits)
print(sum(pred_probab[0]))  # 확률로 확인한다.

y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")



tensor([[-0.0455, -0.0611, -0.0040,  0.0300, -0.0768, -0.1588, -0.1163,  0.0160,
          0.0105,  0.2520]], grad_fn=<AddmmBackward0>)
tensor(1., grad_fn=<AddBackward0>)
Predicted class: tensor([9])


In [64]:
# 3개의 이미지를 생성하여 신경망 통과시에 어떤일이 일어나는지 보기
input_image = torch.rand(3,28,28)
print(input_image.size())

# flatten = nn.Flatten()
# 이미지가 통과하면 784개의 데이터가 하나의 배열이 된다
nn.Flatten()(input_image).size()
flat_image = nn.Flatten()(input_image)

# 선형레이어이다. 가중치와 편향을 사용해 입력 데이터에 선형변환을 한다
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size()) # 784개의 데이터가 선형변환되어 20개가 되었다


torch.Size([3, 28, 28])
torch.Size([3, 20])


In [ ]:
# 렐루 활성화 함수를 적용하여 비선형성을 도입한다

print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

In [72]:
#  Sequential은 순서를 갖는 컨테이너이다. 정의한 순서대로 모듈들을 전달한다
seq_modules = nn.Sequential(
    nn.Flatten(),
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
logits

# class 확률 예측
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

tensor([[-0.0355,  0.0294,  0.1307, -0.2620,  0.1019,  0.1235, -0.1135,  0.2825,
         -0.1903, -0.2889],
        [-0.2235,  0.3350,  0.1465, -0.1457,  0.1916,  0.2084,  0.1010,  0.3395,
         -0.1566, -0.2478],
        [-0.1344,  0.2073,  0.1579, -0.0953,  0.1982,  0.2271, -0.1042,  0.2454,
         -0.2363, -0.1984]], grad_fn=<AddmmBackward0>)

In [73]:
#매개변수들 확인하기

print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0008, -0.0190,  0.0028,  ..., -0.0127,  0.0307,  0.0330],
        [ 0.0348, -0.0093,  0.0199,  ...,  0.0272,  0.0145,  0.0346]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0270, -0.0109], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0135, -0.0242,  0.0160,  ..., -0.0319, -0.0070, -0.0133],
        [ 0.0005, -0.0099,  0.0374,  ..., -0.0144, -0.0006,  0.0385]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 